## Dependence on Russian and Ukrainian Imports

As the world is changing this week, we can try to understand how important Russian and Ukrainian imports are to the US.

[Census.Gov](https://www.census.gov/data/developers/data-sets/international-trade.html) has international trade data exposed for us to download.


In [ ]:
import requests

In [ ]:
base_url = "http://api.census.gov/data/timeseries/intltrade/imports/enduse?"
params = {"get": "CTY_CODE,CTY_NAME,I_ENDUSE,I_ENDUSE_LDESC,GEN_VAL_MO,CON_VAL_MO",
          "time": "2013-01"}
resp = requests.get(base_url, params=params)

In [ ]:
resp.status_code

200

In [ ]:
dat = resp.json()

In [ ]:
type(dat)

list

In [ ]:
len(dat)

13044

In [ ]:
len(dat[10])

7

In [ ]:
dat[:3]

[['CTY_CODE',
  'CTY_NAME',
  'I_ENDUSE',
  'I_ENDUSE_LDESC',
  'GEN_VAL_MO',
  'CON_VAL_MO',
  'time'],
 ['4050',
  'FINLAND',
  '-',
  'TOTAL IMPORTS FOR ALL END-USE CODES',
  '319554327',
  '335786013',
  '2013-01'],
 ['4635',
  'KYRGYZSTAN',
  '-',
  'TOTAL IMPORTS FOR ALL END-USE CODES',
  '17592',
  '17592',
  '2013-01']]

In [ ]:
import pandas as pd

df = pd.DataFrame(dat[1:], columns=dat[0])

In [ ]:
df.shape

(13043, 7)

In [ ]:
df.head(2)

,CTY_CODE,CTY_NAME,I_ENDUSE,I_ENDUSE_LDESC,GEN_VAL_MO,CON_VAL_MO,time
0,4050,FINLAND,-,TOTAL IMPORTS FOR ALL END-USE CODES,319554327,335786013,2013-01
1,4635,KYRGYZSTAN,-,TOTAL IMPORTS FOR ALL END-USE CODES,17592,17592,2013-01


## Get more data

- Get the general imports value for all end-use codes for 2013-2020 for the months 01-12.
- Combine all of the data into one data frame

In [ ]:
# Shrinking the years so Ed doesn't crash
years = list(range(2018, 2021))
months = range(1, 13)

In [ ]:
dfs = []
for year in years:
    print(year)
    for month in months:
        # year = "2020"
        # month = "01"
        month = month if month >= 10 else "0" + str(month)
        base_url = "http://api.census.gov/data/timeseries/intltrade/imports/enduse?"
        params = {"get": "CTY_CODE,CTY_NAME,I_ENDUSE,I_ENDUSE_LDESC,GEN_VAL_MO,CON_VAL_MO",
                  "time": "{YEAR}-{MONTH}".format(YEAR=year, MONTH=month)}
        resp = requests.get(base_url, params=params)
        dat = resp.json()
        df = pd.DataFrame(dat[1:], columns=dat[0])
        dfs.append(df)

2018
2019


In [ ]:
bdf = pd.concat(dfs)

In [ ]:
bdf.tail(3)

,CTY_CODE,CTY_NAME,I_ENDUSE,I_ENDUSE_LDESC,GEN_VAL_MO,CON_VAL_MO,time
19097,3070,VENEZUELA,50040,"OTHER (MOVIES, MISCELLANEOUS IMPORTS, AND SPEC...",0,0,2020-12
19098,3120,GUYANA,50040,"OTHER (MOVIES, MISCELLANEOUS IMPORTS, AND SPEC...",78669,78669,2020-12
19099,3150,SURINAME,50040,"OTHER (MOVIES, MISCELLANEOUS IMPORTS, AND SPEC...",75674,75674,2020-12


##  Analyze the Dependence

Quantify a major dependence we have on Russia or Ukrain using the data we've collected.

In [ ]:
sdf = bdf_grp.get_group(grp)
from_russian = sdf.CTY_NAME == "RUSSIA"


In [ ]:
sdf.shape

(4024, 7)

In [ ]:
russia_val = sdf.loc[from_russian, "GEN_VAL_MO"]

In [ ]:
russia_val.astype(float).sum()

7180941.0

In [ ]:
bdf_grp = bdf.groupby("I_ENDUSE_LDESC")

prop_russia = {}
for grp in bdf_grp.groups:
    sdf = bdf_grp.get_group(grp)
    from_russian = sdf.CTY_NAME == "RUSSIA"
    russia_val = sdf.loc[from_russian, "GEN_VAL_MO"].astype(float).sum()
    prop_russia.update({grp: russia_val / sdf.GEN_VAL_MO.astype(float).sum()})

In [ ]:
sorted([(k, v) for k, v in prop_russia.items()], key=lambda x: x[1])[-10:]

[('NONFERROUS METALS, OTHER', 0.008325050758562219),
 ('OTHER MILITARY EQUIPMENT', 0.008895039279254493),
 ('PLYWOOD AND VENEERS', 0.00986277801523884),
 ('NICKEL', 0.01232060479631816),
 ('CHEMICALS-FERTILIZERS', 0.014133116188763358),
 ('OTHER PRECIOUS METALS', 0.021334977908361304),
 ('STEELMAKING MATERIALS', 0.027548659235540184),
 ('NUCLEAR FUEL MATERIALS', 0.038887036518478385),
 ('FUEL OIL', 0.05804145846671529),
 ('SPACECRAFT, EXCLUDING MILITARY', 0.1342180905408842)]